In [102]:
from matplotlib.pyplot import table
import pandas as pd
from sqlalchemy import create_engine
import os
import psycopg2
from sqlalchemy import create_engine
import datetime
import dateutil
from google_play_scraper import Sort, reviews
from tqdm.auto import tqdm

In [103]:
class DatabaseConnection:
    def __init__(self):
        connection_info = {
            "host": os.environ.get("PG_HOST", "localhost"),
            "port": os.environ.get("PG_PORT", 5432),
            "db": os.environ.get("PG_DATABASE", "playstore"),
            "user": os.environ.get("PG_USER", "postgres"),
            "password": os.environ.get("PG_PASS", "postgres"),
        }

        # Sql engine
        self.db_engine = create_engine(
            "postgresql+psycopg2://{}:{}@{}:{}/{}".format(
                connection_info["user"],
                connection_info["password"],
                connection_info["host"],
                connection_info["port"],
                connection_info["db"],
            )
        )
        # psql engine
        self.db_conn = psycopg2.connect(
            host=connection_info["host"],
            port=connection_info["port"],
            database=connection_info["db"],
            user=connection_info["user"],
            password=connection_info["password"],
        )

In [150]:
# Create function get PS Review based on date
app_id = "id.co.bri.brimo"
app_id = "net.myinfosys.PermataMobileX"
conn = DatabaseConnection().db_conn
MAX_COUNT_EACH_FETCH = 200

In [151]:
# helper.status


def get_last_date_db(app_id, conn=None):
    # create query
    date_query = f"""
        SELECT MAX(at)
        FROM review
        WHERE apps = '{app_id}'
        """

    # get dttm from db
    curr = conn.cursor()

    # execute query
    curr.execute(date_query)
    t = curr.fetchall()[0][0]
    if not isinstance(t, datetime.date) and t:
        t = dateutil.parser.parse(t)
    if not t:
        t = dateutil.parser.parse("2023-02-01")
    return t

In [152]:
continuation_token = None
result = []
condition = False
last_date = get_last_date_db(app_id, conn)

In [153]:
def get_review_by_last_date(app_id, last_date) -> pd.DataFrame:
    results = []
    continuation_token = None
    is_date_match = False

    pbar = tqdm()
    while is_date_match is False:
        try:
            result, continuation_token = reviews(
                app_id,
                count=MAX_COUNT_EACH_FETCH,
                continuation_token=continuation_token,
                lang="id",  # defaults to 'en'
                country="id",  # defaults to 'us'
                sort=Sort.NEWEST,  # defaults to Sort.MOST_RELEVANT
                filter_score_with=None,  # defaults to None(means all score)
            )
            results += result
            results_date = results[-1]["at"]
            pbar.update(len(results))
        except:
            continue

        if len(results) % 10000 == 0:
            print(results_date)

        if results[-1]["at"] <= last_date:
            is_date_match = True

    df = pd.DataFrame(result)
    return df

In [154]:
last_date = dateutil.parser.parse("2023-02-01")

In [155]:
df = get_review_by_last_date(app_id, last_date)

0it [00:00, ?it/s]

In [157]:
df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,7071def4-1d0e-4eac-a841-42d46e712298,Erza Wahyu,https://play-lh.googleusercontent.com/a/AEdFTp...,Aplikasi perbankan yg mudah di operasikan dan ...,5,1,1.43.2,2023-02-03 12:27:40,Gunakan selalu PermataMobile X untuk transaksi...,2023-02-03 12:47:22
1,e51cc932-fa1a-404a-a135-84f791ab6b20,Hartono Tono,https://play-lh.googleusercontent.com/a/AEdFTp...,Tidak perlu ribet kecabang semua transaksi bis...,5,2,1.43.1,2023-02-03 12:27:37,Senang sekali atas ulasan yang Pak Hartono ber...,2023-02-03 12:47:00
2,a1bea3f5-9bb4-4fff-a1c7-05fc0497c009,Irfan Trias Wanto Trias Wanto,https://play-lh.googleusercontent.com/a/AEdFTp...,Sangat terbantu dengan aplikasi ini..,5,1,1.43.2,2023-02-03 12:26:42,Bangga dapat membantu dan mempermudah transaks...,2023-02-03 12:46:28
3,73139872-23cb-4c41-9559-c8769445e4d4,Wahyudi Jusela,https://play-lh.googleusercontent.com/a/AEdFTp...,"Sangat2 bermanfaat, mempermudah dalam layanan ...",5,2,1.43.2,2023-02-03 12:25:27,Senang melihat Pak Wahyudi terbantu menggunaka...,2023-02-03 12:46:10
4,4d881369-2aeb-4796-88e3-59618826a941,Cedrik Anis,https://play-lh.googleusercontent.com/a/AEdFTp...,Aplikasi yg sangat membantu dalam bertransaksi...,5,1,1.43.1,2023-02-03 12:24:08,Hi Pak/Ibu Cedrik. Jangan lupa ajak teman-tema...,2023-02-03 12:45:44
...,...,...,...,...,...,...,...,...,...,...
195,d5afc0b0-f72e-4db9-adce-89c0bc77d642,Dermawan wawan,https://play-lh.googleusercontent.com/a/AEdFTp...,Pakai permata mobile x makin lebih mudah untuk...,5,0,None,2023-01-31 21:00:14,Hi Pak Dermawan Wawan. Bangga dapat membantu d...,2023-01-31 22:03:31
196,88128af7-1621-49d7-b047-1e73912b62dd,HERI CHANNEL,https://play-lh.googleusercontent.com/a-/AD5-W...,"Sangat membantu,,good permata bank",5,0,1.43.1,2023-01-31 20:59:59,Hi Pak Heri Channel. Gunakan selalu PermataMob...,2023-01-31 22:02:21
197,39db369b-aacb-4eb6-82ce-3b2cdbd22ec5,ntha yunita,https://play-lh.googleusercontent.com/a/AEdFTp...,Aplikasi keren.. yg sangat membantu dalam bert...,5,0,1.43.1,2023-01-31 20:57:06,Semoga semakin mempermudah transaksi Perbankan...,2023-01-31 22:43:31
198,6b3c3c6c-1318-46cf-879d-989aadb18e21,Ismail,https://play-lh.googleusercontent.com/a-/AD5-W...,Aplikasi yang sangat bagus... mempermudah sega...,5,0,1.43.1,2023-01-31 20:54:16,Hi Pak Ismail. Terimakasih sudah pakai Permata...,2023-01-31 21:59:25


In [113]:
results = []
continuation_token = None
is_date_match = False

pbar = tqdm()
while is_date_match is False:
    result, continuation_token = reviews(
        app_id,
        count=MAX_COUNT_EACH_FETCH,
        continuation_token=continuation_token,
        lang="id",  # defaults to 'en'
        country="id",  # defaults to 'us'
        sort=Sort.NEWEST,  # defaults to Sort.MOST_RELEVANT
        filter_score_with=None,  # defaults to None(means all score)
    )
    results += result
    results_date = results[-1]["at"]
    pbar.update(len(results))

    if len(results) % 1000 == 0:
        print(results_date)

    if results[-1]["at"] <= last_date:
        is_date_match = True

0it [00:00, ?it/s]

2023-02-05 15:35:22
2023-02-03 10:38:12
2023-02-01 17:38:01
2023-01-31 01:20:22
2023-01-28 21:02:09
2023-01-27 07:02:23
2023-01-25 13:04:21
2023-01-23 17:05:43
2023-01-21 11:57:59
2023-01-19 15:38:47
2023-01-17 15:44:39
2023-01-15 13:50:31
2023-01-13 11:56:48
2023-01-11 14:44:36
2023-01-09 21:20:06
2023-01-08 02:06:22
2023-01-06 12:11:55
2023-01-04 22:14:59
2023-01-03 14:58:16
2023-01-02 08:50:59


In [71]:
# -1 adalah tanggal yang paling lama , idx 1 adalah tanggal terbaru
results[-1]["at"] <= last_date

True